In [1]:
import os
import pandas as pd
import altair as alt

In [2]:
processed_data_path = os.path.join("..", "data", "processed")
hardware_files = [f for f in os.listdir(processed_data_path) if "hardware" in f]
df_hardware = pd.concat([pd.read_csv(os.path.join(processed_data_path, f), parse_dates=["Date"]) for f in hardware_files])
df_hardware

,Product,Group,Date,Qty
0,2DS,2DS/3DS,2011-03-31,0.00
1,2DS,2DS/3DS,2012-03-31,0.00
2,2DS,2DS/3DS,2013-03-31,0.00
3,2DS,2DS/3DS,2014-03-31,2.20
4,2DS,2DS/3DS,2015-03-31,1.55
...,...,...,...,...
8,PS4,PS4,2022-03-31,1.00
9,PS5,PS5,2021-03-31,7.80
10,PS5,PS5,2022-03-31,11.50
11,PS5,PS5,2023-03-31,19.10


In [3]:
df_ranked = df_hardware.copy()
df_ranked = df_ranked[df_ranked.Qty > 0]
df_ranked = df_ranked.sort_values(["Product", "Date"], ascending=[True, True])
# dense rank Date values by Group column
df_ranked["rk_date"] = df_ranked.groupby("Product")["Date"].rank(method="dense", ascending=True).astype(int)
df_ranked

,Product,Group,Date,Qty,rk_date
3,2DS,2DS/3DS,2014-03-31,2.20,1
4,2DS,2DS/3DS,2015-03-31,1.55,2
5,2DS,2DS/3DS,2016-03-31,1.17,3
6,2DS,2DS/3DS,2017-03-31,2.50,4
7,2DS,2DS/3DS,2018-03-31,1.99,5
...,...,...,...,...,...
202,Wii U,Wii U,2013-03-31,3.45,1
203,Wii U,Wii U,2014-03-31,2.73,2
204,Wii U,Wii U,2015-03-31,3.38,3
205,Wii U,Wii U,2016-03-31,3.25,4


In [6]:
selection = alt.selection_point(fields=['Group'], bind='legend')
opacity = alt.condition(selection, alt.value(1), alt.value(0.2))

alt.Chart(df_ranked).mark_line().encode(
    x=alt.X("rk_date:N", title="Ranked Year"),
    y=alt.Y("Qty:Q", title="Units Sold (Millions)"),
    color=alt.Color("Product:N", title="Product"),
    tooltip=["Product", "Date", "Qty"],
    opacity=opacity)\
.add_params(selection)\
.properties(
    title="Hardware Sales by Product",
    width=800,
    height=400
).interactive()

alt.Chart(...)

In [9]:
set(df_hardware["Product"]), set(df_hardware["Group"])

({'2DS',
  '3DS',
  '3DS XL',
  'DS',
  'DS Lite',
  'DSi',
  'DSi XL',
  'Gamecube',
  'New 2DS XL',
  'New 3DS',
  'New 3DS XL',
  'PS4',
  'PS5',
  'Switch',
  'Switch Lite',
  'Switch – OLED Model',
  'Wii',
  'Wii U'},
 {'2DS/3DS', 'DS', 'Gamecube', 'PS4', 'PS5', 'Switch', 'Wii', 'Wii U'})

In [10]:
df_regrouped = df_hardware.copy()
df_regrouped = df_regrouped.groupby(["Group", "Date"])["Qty"].sum().reset_index()

In [11]:
df_ranked = df_regrouped.copy()
df_ranked = df_ranked[df_ranked.Qty > 0]
df_ranked = df_ranked.sort_values(["Group", "Date"], ascending=[True, True])
# dense rank Date values by Group column
df_ranked["rk_date"] = df_ranked.groupby("Group")["Date"].rank(method="dense", ascending=True).astype(int)
df_ranked

,Group,Date,Qty,rk_date
0,2DS/3DS,2011-03-31,3.61,1
1,2DS/3DS,2012-03-31,13.52,2
2,2DS/3DS,2013-03-31,13.96,3
3,2DS/3DS,2014-03-31,12.24,4
4,2DS/3DS,2015-03-31,8.74,5
...,...,...,...,...
82,Wii U,2013-03-31,3.45,1
83,Wii U,2014-03-31,2.73,2
84,Wii U,2015-03-31,3.38,3
85,Wii U,2016-03-31,3.25,4


In [13]:
consoles = ["DS", "Switch", "PS4", "PS5"]
# consoles = ["DS", "2DS/3DS", "Gamecube", "Wii", "Wii U", "Switch"]
filter_rows = df_ranked["Group"].isin(consoles)
df_plot = df_ranked[filter_rows]
df_plot = df_plot[df_plot["Date"].dt.year <= 2023]

selection = alt.selection_point(fields=['Group'], bind='legend')
opacity=alt.condition(selection, alt.value(1), alt.value(0.2))

width = 500
height = 500
chart = alt.Chart(df_plot).mark_line(point={"shape": "square", "size": 50})\
    .encode(
        x=alt.X("rk_date:N", title="Ranked Year"),
        y=alt.Y("Qty:Q", title="Units Sold (Millions)"),
        color=alt.Color("Group:N", title="Group").sort(consoles),
        opacity=opacity,
        tooltip=["Group", "Date", "Qty"]
        
)

# width = 100
# height = 100
# chart = alt.Chart(df_plot).mark_bar()\
#     .encode(
#         x=alt.X("Group:N", title="Group"),
#         y=alt.Y("Qty:Q", title="Units Sold (Millions)"),
#         color=alt.Color("Group:N", title="Group"),
#         column=alt.Color("rk_date:N", title="Ranked Year"),
#         opacity=opacity,
#         tooltip=["Group", "Date", "Qty"]
# )

chart = chart.add_params(selection)
chart = chart.properties(
    title="Hardware Sales by Group",
    width=width,
    height=height
).interactive()

chart

alt.Chart(...)

In [14]:
chart.save("../figs/Nintendo_vs_Playstation_ranked_year_model_sales.png")

In [47]:
consoles_order = ["DS", "2DS/3DS", "Gamecube", "Wii", "Wii U", "Switch"]
consoles_to_remove = ["PS4", "PS5"]

# consoles_order = ["PS4", "PS5"]
# consoles_to_remove = ["DS", "2DS/3DS", "Gamecube", "Wii", "Wii U", "Switch"]

df_filtered = df_hardware.copy()
df_filtered = df_filtered[~df_filtered["Group"].isin(consoles_to_remove)]
df_filtered = df_filtered[df_filtered.Qty >= 0.1]
df_filtered = df_filtered[df_filtered["Date"].dt.year <= 2023]

# df_filtered = df_filtered.groupby(["Group", "Date"])["Qty"].sum().reset_index()
df_filtered = df_filtered.groupby(["Group", "Date"])["Qty"].sum().reset_index()

fy_totals = df_filtered.groupby("Date")["Qty"].sum().reset_index()

# plot altair chart
selection = alt.selection_point(fields=['Group'], bind='legend')
opacity=alt.condition(selection, alt.value(1), alt.value(0.2))

width = 500
height = 500
chart = alt.Chart(df_filtered).mark_line(point={"shape": "square", "size": 50})\
    .encode(
        x=alt.X("Date:T", title="Date"),
        y=alt.Y("Qty:Q", title="Units Sold (Millions)"),
        color=alt.Color("Group:N", title="Group").sort(consoles_order),
        opacity=opacity,
        tooltip=["Group", "Date", "Qty"]
    )

chart = chart.add_params(selection)

fiscal_year_totals_chart = alt.Chart(fy_totals)\
    .mark_line(color='black', strokeDash=[5, 5], point={"shape": "square", "size": 50, "color": "black"})\
        .encode(
            x=alt.X("Date:T", title="Date"),
            y=alt.Y("Qty:Q", title="Units Sold (Millions)")
            )

chart += fiscal_year_totals_chart

chart = chart.properties(
    title="Hardware Sales by Group",
    width=width,
    height=height
).interactive()
chart

alt.LayerChart(...)

In [48]:
chart.save("../figs/Nintendo_Models_vs_total.png")

In [49]:
# plot altair chart
selection = alt.selection_point(fields=['Group'], bind='legend')
opacity=alt.condition(selection, alt.value(1), alt.value(0.2))

width = 500
height = 500
chart = alt.Chart(df_filtered).mark_line(point={"shape": "square", "size": 50})\
    .transform_window(
    # Sort the data chronologically
    sort=[{'field': 'Date'}],
    # Include all previous records before the current record and none after
    # (This is the default value so you could skip it and it would still work.)
    frame=[None, 0],
    # group by
    groupby=["Group"],
    # What to add up as you go
    cumulative_qty='sum(Qty)'
)\
    .encode(
        x=alt.X("Date:T", title="Date"),
        y=alt.Y("cumulative_qty:Q", title="Cum. Sum Units Sold (Millions)"),
        color=alt.Color("Group:N", title="Group").sort(consoles_order),
        opacity=opacity,
        tooltip=["Group", "Date", alt.Tooltip("cumulative_qty:Q", title="Cum. Sum Units Sold (Millions)")]
    )

chart = chart.add_params(selection)

fiscal_year_totals_chart = alt.Chart(fy_totals)\
    .mark_line(color='black', strokeDash=[5, 5], point={"shape": "square", "size": 50, "color": "black"})\
    .transform_window(
    # Sort the data chronologically
    sort=[{'field': 'Date'}],
    # Include all previous records before the current record and none after
    # (This is the default value so you could skip it and it would still work.)
    frame=[None, 0],
    # What to add up as you go
    cumulative_qty='sum(Qty)'
)\
        .encode(
            x=alt.X("Date:T", title="Date"),
            y=alt.Y("cumulative_qty:Q", title="Cum. Sum Units Sold (Millions)"),
            tooltip=["Date", alt.Tooltip("cumulative_qty:Q", title="Cum. Sum Units Sold (Millions)")],
            )

chart += fiscal_year_totals_chart

chart = chart.properties(
    title="Cumulative Hardware Unit Sales by Group",
    width=width,
    height=height
).interactive()
chart

alt.LayerChart(...)

In [50]:
df_filtered = df_filtered.sort_values(["Group", "Date"], ascending=[True, True])
df_filtered["cumsum"] = df_filtered.groupby("Group")["Qty"].cumsum()

fy_totals = fy_totals.sort_values(["Date"], ascending=[True])
fy_totals["cumsum"] = fy_totals["Qty"].cumsum()

# pct change of cumsum
df_filtered["pct_change"] = df_filtered.groupby("Group")["cumsum"].pct_change()
fy_totals["pct_change"] = fy_totals["cumsum"].pct_change()	


# plot altair chart

# plot altair chart
selection = alt.selection_point(fields=['Group'], bind='legend')
opacity=alt.condition(selection, alt.value(1), alt.value(0.2))

width = 500
height = 500
chart = alt.Chart(df_filtered).mark_line(point={"shape": "square", "size": 50})\
    .encode(
        x=alt.X("Date:T", title="Date"),
        y=alt.Y("pct_change:Q", title="Pct. Change in Cum. Sum Units Sold (Millions)").axis(format='%'),
        color=alt.Color("Group:N", title="Group").sort(consoles_order),
        opacity=opacity,
        tooltip=["Group", "Date", alt.Tooltip("pct_change:Q", title="% change in Cum. Sum Units Sold (Millions)").format("%")]
    )

chart = chart.add_params(selection)

fiscal_year_totals_chart = alt.Chart(fy_totals)\
    .mark_line(color='black', strokeDash=[5, 5], point={"shape": "square", "size": 50, "color": "black"})\
        .encode(
            x=alt.X("Date:T", title="Date"),
            y=alt.Y("pct_change:Q", title="Pct. Change in Cum. Sum Units Sold (Millions)").axis(format='%'),
            tooltip=["Date", alt.Tooltip("pct_change:Q", title="% change in Cum. Sum Units Sold (Millions)").format("%")],
            )

chart += fiscal_year_totals_chart

chart = chart.properties(
    title="Pct. Change in Cumulative Hardware Unit Sales by Group",
    width=width,
    height=height
).interactive()
chart

alt.LayerChart(...)